In [1]:
import pandas as pd
import numpy as np

# Preparation

In [3]:
data = []
with open ("../kaggle-competition/train.txt", "r") as ftxt, \
    open ("../kaggle-competition/train_label.txt", "r") as flabel :
    for t, l in zip(ftxt, flabel):
        t = t.strip()
        l = l.strip()

        data.append({
            "txt": t,
            "label": l
        })
df_train = pd.DataFrame(data)

In [4]:
def sampling_class(df, cls_name, n=20, seed=71):
    
    df_class = df_train[df_train.label==cls_name]
    
    total = len(df_class)
    np.random.seed(seed)
    indices = np.random.permutation(total)[:n]
    return df_class.iloc[indices].txt.values

In [5]:
seeds = dict(zip(['neg', 'neu', 'pos', 'q'], range(4)))

n = 40
sampling_data = []

for k, v in seeds.items():
    txts = sampling_class(df_train, k, n=n, seed=v)
    for t in txts:
        t = t.replace("|", "\|")
        sampling_data.append(dict(
            label=k,
            raw=t,
            tokenised=t
        ))
        

pd.DataFrame(sampling_data).to_csv("./sampling-%d.csv" % n, header=True, index=False)
#             note = """
# ################
# ### label: %s
# ### raw: %s
# ################
#             """ % (k, t)
#             f.write("\n%s\n" % note.strip())
#             f.write("::> %s\n" % t)

# # ab = sampling_class(df_train, "neu")

In [6]:
set(df_train.label.values)

{'neg', 'neu', 'pos', 'q'}

# Postprocessing

Google Spreadsheet: https://docs.google.com/spreadsheets/d/1F_qT33T2iy0tKbflnVC8Ma-EoWEHimV3NmNRgLjN00o/edit#gid=1302375309

In [1]:
filepath = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRm-f8qstNhxICHzEfhbCacJNQSAZptP-6ockKwsxyck5vtl7e1-A2726Qj2hgp4Oht7WfcbdivQNPT/pub?gid=1302375309&single=true&output=csv"

In [4]:
df = pd.read_csv(filepath)
print("we have %d samples" % len(df))

we have 1000 samples


In [11]:
should_removed = ~df.label.apply(lambda x: len(x.split("-")) > 1)

In [12]:
df_filtered = df[should_removed]

In [13]:
print("we have %d after samples" % len(df_filtered))

we have 993 after samples


In [14]:
filename = "wisesight-%d-samples-tokenised.txt" % len(df)

In [15]:
with open(filename, "w") as ft, open(filename.replace(".txt", ".label"), "w") as fl:
         for l in df_filtered.tokenised.values:
            l = l.strip()
            ft.write("%s\n" % l.replace("|", ""))
            fl.write("%s\n" % l)